In [3]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
import glob
from keras.models import Sequential
from keras.layers import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tqdm import tqdm_notebook as tq
from random import shuffle
import os
from keras.optimizers import *
from shutil import copyfile

%matplotlib inline


We need to move boats around.

In [4]:
def group_boats(groups, new_folder = "./boats_id_train/"):
    files_list = glob.glob('./boats_id/*/*/*.jpg')
    tuples = [x.split("\\") for x in files_list]
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)
        
    for i in range(len(groups) + 1):
        if not os.path.exists(new_folder + str(i) + "/"):
            os.makedirs(new_folder + str(i) + "/")
    
    moved = [False for _ in tuples]
    
    for i, group in enumerate(tq(groups)):
        if type(group) is int:
            group  = [group]
        
        for folder_idx in group:
            for j, tup in enumerate(tuples):
                if int(tup[2]) == folder_idx:
                    copyfile(files_list[j], new_folder + str(i + 1) + "/" + tup[-1])
                    moved[j] = True
    
    for j, move in enumerate(tq(moved)):
        if not move:
            copyfile(files_list[j], new_folder + "0" + "/" + tuples[j][-1])
    

In [23]:
group_boats([[0,1],[2,3],4,5,6,[7,8],[9,10],11,12,13,[14,15],20,22,36,26,27,32,42])

Now we need to have the same number of pictures in each class. So we'll add new ones.

In [54]:
def create_new_pictures(train_folder, class_folder, generator,total_images, 
                        batch_size = 32, target_size = (256,256), color_mode = "rgb"):
    #count = len(os.listdir(train_folder + "/" + class_folder))
    count = 0  
    gen_folder = train_folder + "_gen/"
        
    if not os.path.exists(gen_folder):
        os.makedirs(gen_folder)
        
    if not os.path.exists(gen_folder + class_folder):
        os.makedirs(gen_folder + class_folder)
        
    
    progbar = tq(total = total_images, leave=False)
    progbar.update(count)
    
    
    for X in generator.flow_from_directory(train_folder, target_size=target_size,
                                             classes=[class_folder], 
                                             save_to_dir=train_folder + "_gen/" + class_folder, 
                                             batch_size=batch_size, color_mode=color_mode):
        count +=batch_size
        progbar.update(batch_size)
        if count >= total_images:
            break
    progbar.close()
    

In [83]:
my_generator = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=0.,
    width_shift_range=0.,
    height_shift_range=0.,
    shear_range=0.,
    zoom_range=0.,
    channel_shift_range=200,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    noise = 0)

In [84]:
my_test_generator = ImageDataGenerator()

In [85]:
create_new_pictures("New Folder", "examples", my_generator,20, 1, (None,None))

Found 38 images belonging to 1 classes.


In [ ]:
for string in os.listdir("boats_id_train"):
    create_new_pictures("boats_id_train", string, my_generator,500, 32, (128,128))

In [8]:
def equilibrate_dataset(folder, nb_images_per_class, generator):
    c = 0
    while c < nb_images_per_class:
        for string in os.listdir(folder):
            create_new_pictures(folder, string, generator, 200, 1, (64,64))
        c += 200

In [10]:
equilibrate_dataset("boats_id_train - Copy", 400,my_generator)

Found 1117 images belonging to 1 classes.
Found 856 images belonging to 1 classes.
Found 31 images belonging to 1 classes.
Found 34 images belonging to 1 classes.
Found 31 images belonging to 1 classes.
Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 51 images belonging to 1 classes.
Found 54 images belonging to 1 classes.
Found 83 images belonging to 1 classes.
Found 214 images belonging to 1 classes.
Found 46 images belonging to 1 classes.
Found 88 images belonging to 1 classes.
Found 235 images belonging to 1 classes.
Found 542 images belonging to 1 classes.
Found 39 images belonging to 1 classes.
Found 245 images belonging to 1 classes.
Found 1117 images belonging to 1 classes.
Found 856 images belonging to 1 classes.
Found 31 images belonging to 1 classes.
Found 34 images belonging to 1 classes.
Found 31 images belonging to 1 classes.
Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 cl

In [9]:
equilibrate_dataset("boats_id_test", 200 , my_test_generator)

Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 5 images belonging 